In [1]:
from ModelClass import RegressionModel
from DatasetClass import Dataset, DatasetMass
import matplotlib.pyplot as plt
import numpy as np
import itertools
from src.helpers import make_filter_slice
import tensorflow as tf
import sys
sys.path.append("/home/kvake/.local/lib/python3.11/site-packages")
import optuna
sys.path.append("/home/kvake/.local/lib/python3.9/site-packages")  # Adjust if needed
import optuna_dashboard

2025-03-08 19:14:13.894903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-08 19:14:13.916125: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-08 19:14:13.923521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 19:14:13.944508: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-08 19:14:19.314733: E external/local_xla/xla/

In [2]:
erik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*H125*.root"
patrik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*Ztt*.root"

In [8]:
dataset = Dataset(file_paths=erik_data, file_name = "erik_data")
# dataset.build_dataset()
# dataset.save_data()
dataset.load_data()

print(len(dataset.train_dataset))

#This block of code iterates through the dataset and extracts the pt values of the labels and stores them in a list
data = [labels.numpy()[0] for features, labels in dataset.train_dataset.take(100000)]

# plt.hist(data, bins=100, range=(50, 130), histtype='step', label='pt distribution', density=False)
# plt.legend(loc='upper right')
# plt.title('pt distribution of the dataset')
# plt.xlabel('pt')
# plt.ylabel('Number of events')
# plt.show()

603636


2025-03-08 19:12:41.699125: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
import optuna
from optuna_dashboard import run_server
from DatasetClass import DatasetPt
from ModelClass import RegressionModel
import threading

# ======================
# 1. Build/Load Dataset (Outside Optuna Loop for Efficiency)
# ======================
dataset = DatasetPt(file_paths="erik_data", file_name="erik_data")
dataset.load_data()

print("Train dataset size:", len(dataset.train_dataset))

# ============================
# 2. Define Optuna Objective (Fast Training)
# ============================
def objective(trial):
    """
    Optuna objective function for hyperparameter search.
    """

    model = RegressionModel(
        dataset,
        n_layers             = trial.suggest_int('n_layers', 1, 5),
        hidden_layer_size    = trial.suggest_int('hidden_layer_size', 32, 512),
        initial_learning_rate= trial.suggest_float('initial_learning_rate', 1e-4, 1e-2, log=True),
        n_epochs             = trial.suggest_int('n_epochs', 5, 30, step=5),
        activation_function  = 'relu',
        batch_size           = trial.suggest_int('batch_size', 32,128,step=32)
    )

    model.prepare_dataset()
    model.create_normalizer()
    model.build_model()
    model.train_model(model.train_batch, model.val_batch)

    val_loss, _ = model.model.evaluate(model.val_batch, verbose=0)
    return val_loss

# ========================
# 3. Run the Optuna Study
# ========================
study = optuna.create_study(storage="sqlite:///optuna.db", study_name="Higgs_analysis_pt", direction='minimize')

# ========================
# 4. Start `optuna-dashboard` in a Separate Thread
# ========================

def run_dashboard():
    server = run_server("sqlite:///optuna.db", host="0.0.0.0", port=8080)
    server.run()  # Run dashboard continuously in a separate thread

dashboard_thread = threading.Thread(target=run_dashboard)
dashboard_thread.daemon = True  # Ensures it stops when the script ends
dashboard_thread.start()

def run_optuna():
    study.optimize(objective, n_trials=10, n_jobs=1)

# Run Optuna in a separate thread so the dashboard can be accessed in real-time
optuna_thread = threading.Thread(target=run_optuna)
optuna_thread.start()

# ========================
# 5. Print Results
# ========================
optuna_thread.join()  # Wait for Optuna to finish before printing results
print("Number of finished trials:", len(study.trials))
print("Best trial:", study.best_trial.params)

# Dashboard will keep running in background


Train dataset size: 603636


[I 2025-03-08 19:20:45,321] A new study created in RDB with name: Higgs_analysis_pt
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8080/
Hit Ctrl-C to quit.



Batching datasets...


2025-03-08 19:21:01.233191: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Building model...
Training model...
Epoch 1/30
4716/4716 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 5945.5923 - mean_squared_error: 5945.5923 - val_loss: 323.4619 - val_mean_squared_error: 323.4619
Epoch 2/30
4716/4716 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 313.0353 - mean_squared_error: 313.0353 - val_loss: 297.0313 - val_mean_squared_error: 297.0313
Epoch 3/30
4716/4716 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - loss: 291.1248 - mean_squared_error: 291.1248 - val_loss: 285.0720 - val_mean_squared_error: 285.0720
Epoch 4/30
4716/4716 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 281.0669 - mean_squared_error: 281.0669 - val_loss: 278.6642 - val_mean_squared_error: 278.6642
Epoch 5/30
4716/4716 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 275.1577 - mean_squared_error: 275.1577 - val_loss: 274.2500 - val_mean_squared_error: 274.2500
Epoch 6/30
4716/4716 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 270.9183 - mean_squared_error: 270.9184 - val_loss: 270.5456 - val_mean_squared_error: 270.5456
Epoch 7/

...

[I 2025-03-08 20:20:31,481] Trial 8 finished with value: 230.46766662597656 and parameters: {'n_layers': 1, 'hidden_layer_size': 502, 'initial_learning_rate': 0.0024490315446469044, 'n_epochs': 5, 'batch_size': 32}. Best is trial 5 with value: 190.13284301757812.


Batching datasets...
Building model...
Training model...
Epoch 1/25
18864/18864 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 14310.4199 - mean_squared_error: 14310.4199 - val_loss: 405.1011 - val_mean_squared_error: 405.1011
Epoch 2/25
18864/18864 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 377.4053 - mean_squared_error: 377.4053 - val_loss: 335.9081 - val_mean_squared_error: 335.9081
Epoch 3/25
18864/18864 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - loss: 331.9269 - mean_squared_error: 331.9269 - val_loss: 318.6389 - val_mean_squared_error: 318.6389
Epoch 4/25
18864/18864 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 316.3176 - mean_squared_error: 316.3176 - val_loss: 307.0570 - val_mean_squared_error: 307.0570
Epoch 5/25
18864/18864 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 305.2882 - mean_squared_error: 305.2882 - val_loss: 298.9324 - val_mean_squared_error: 298.9324
Epoch 6/25
18864/18864 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 297.5993 - mean_squared_error: 297.5993 - val_loss: 293.5770 - val_me

[I 2025-03-08 20:29:07,126] Trial 9 finished with value: 274.10595703125 and parameters: {'n_layers': 1, 'hidden_layer_size': 82, 'initial_learning_rate': 0.00012687052050364302, 'n_epochs': 25, 'batch_size': 32}. Best is trial 5 with value: 190.13284301757812.


Number of finished trials: 10
Best trial: {'n_layers': 4, 'hidden_layer_size': 355, 'initial_learning_rate': 0.0018702419332025772, 'n_epochs': 20, 'batch_size': 128}


127.0.0.1 - - [08/Mar/2025 20:30:50] "GET / HTTP/1.1" 302 0
